### Import

In [33]:
import ebooklib
import re
from bs4 import BeautifulSoup
from ebooklib import epub

### epub2html

In [34]:
def epub2thtml(epub_path):
    book = epub.read_epub(epub_path)
    chapters = []
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            chapters.append(item.get_content())
    return chapters

### Element Blacklist

In [35]:
blacklist = [   '[document]',   'noscript', 'header',   'html', 'meta', 'head','input', 'script',   ]

#### Chapter to text

In [36]:
def chap2text(chap):
    output = ''
    soup = BeautifulSoup(chap, 'html.parser')
    text = soup.find_all(text=True)
    for t in text:
        if t.parent.name not in blacklist:
            output += '{} '.format(t)
    return output

### thtml2ttext

In [37]:
def thtml2ttext(thtml):
    Output = []
    for html in thtml:
        text =  chap2text(html)
        Output.append(text)
    return Output

### Main Function

In [38]:
def epub2text(epub_path):
    chapters = epub2thtml(epub_path)
    ttext = thtml2ttext(chapters)
    return ttext

### Convert

In [39]:
out = epub2text('../../data/books/principles_of_neuroscience.epub')

### Clean all chapters and write them to separate files

In [96]:
for c,chapter in enumerate(out):

    #Remove Everything After Selected Readings
    chapter = chapter.lower()
    
    if ('selected readings' in chapter):
        index = chapter.index('selected readings')
        chapter = chapter[:index-1]
        chapter = chapter.replace(u'\xa0', u' ')
        chapter = chapter.replace(u'\n', u'.')
        chapter = re.sub(r'[^a-zA-Z.]',' ', chapter)
#         chapter = chapter.replace('  .', u'.')
#         chapter = chapter.replace(' .', u'.')
#         chapter = chapter.replace('..', u'.')
        words = chapter.split(' ')
        words = [word for word in words if len(word) > 0]
        chapter = ' '.join(words)
        chapter = chapter.replace(' .', u'.')
        chapter = chapter.split('.')
        with open('../../data/books/principles_of_neuroscience_chapter_' + str(c) + '.txt','w') as f:
            for line in chapter:
                if (len(line) > 10):
                    if (line[0] == ' '):
                        line = line[1:]
                    f.write(line + '\n')